In [1]:
import geopandas as gpd
import numpy as np
import shapely.geometry

/home/luckyw0w/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [8]:
gdf = gpd.read_file("base_geojson/TL_state_shapefile_for_clip.geojson")  #Given location of shapefile of which grid has to be make

STEP = 10000 # size of one cube of grid (in metres)
crs = gdf.estimate_utm_crs() # EPSG:32644 - NOTE:- CONVERT INTO DEGREE COORDINATE SYSTEM TO METRIC IF NOT

In [9]:
a, b, c, d = gdf.to_crs(crs).total_bounds

# create a grid for geometry
gdf_grid = gpd.GeoDataFrame(
    geometry=[
        shapely.geometry.box(minx, miny, maxx, maxy)
        for minx, maxx in zip(np.arange(a, c, STEP), np.arange(a, c, STEP)[1:])
        for miny, maxy in zip(np.arange(b, d, STEP), np.arange(b, d, STEP)[1:])
    ],
    crs=crs,
).to_crs(gdf.crs)

# exclude geometries that cross antimeridian 
gdf_grid = gdf_grid.loc[~gdf_grid["geometry"].bounds.pipe(lambda d: d["maxx"] - d["minx"]).ge(350)]

# restrict grid to only squares that intersect with geometry
gdf_grid = (
    gdf_grid.sjoin(gdf.dissolve().loc[:,["geometry"]])
    .pipe(lambda d: d.groupby(d.index).first())
    .set_crs(gdf.crs)
    .drop(columns=["index_right"])
)
m = gdf.explore(color="red", style_kwds={"fillOpacity":0})
gdf_grid.explore(m=m)

In [11]:
#SAVE GRID
gdf_grid.to_file('base_geojson/Telangana_grid10km.geojson', driver='GeoJSON')